In [1]:
#!/usr/bin/env python
# coding: utf-8

Starting Spark application


ID,YARN Application ID,Kind,State,Spark UI,Driver log,User,Current session?
7,application_1693171143167_0008,pyspark,idle,Link,Link,None,✔


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

SparkSession available as 'spark'.


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [2]:
import os
import sys
import pytz
import numpy as np
import datetime
from pyspark.sql import SparkSession
from pyspark.sql import SQLContext
from pyspark.sql.functions import udf, lpad, translate
from datetime import datetime 
from datetime import timedelta
from datetime import date
from dateutil.relativedelta import relativedelta
from pyspark.sql.types import *
from pyspark.sql.functions import count, avg, col

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [3]:
### FREQ FUNCTION
def Freq(pTabela,pColuna):
 
    qtd_total=pTabela.count()
   
    pTabela.registerTempTable("tab_input")
    frq = spark.sql(    
            """
                select
                    {col},
                    count(*) as qtd_absoluto,
                    round(100*(count(*) / {tot}),2) as qtd_percentual
                from
                    tab_input
                group by
                    {col}
                order by
                    {col}
            """.format(col=pColuna, tot=qtd_total))
       
    qtd=frq.count()
    print('Quantidade de dominios',qtd)
    if qtd > 500:
        frq.show(1000,truncate=False)
        return "Dominio muito granular"
    
    else:
        frq.show(qtd,truncate=False)
        print("volumetria total:",qtd_total)
        return 'Freq da coluna ' + pColuna;

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [4]:
agora=datetime.now(pytz.timezone('America/Sao_Paulo'))
dthproc=agora.strftime("%Y%m%d%H%M%S")

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [5]:
dthproc

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

'20230827221236'

In [6]:
### CHANGE FOR EACH FILE
spark = SparkSession.builder.appName('user_id_dim').getOrCreate()
sqlContext=SQLContext(spark.sparkContext)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [7]:
### CHANGE FOR EACH FILE
raw ='t_user_info'

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [8]:
bucket_control = "bkt-musicstream-bi/Files/Control"
bucket_ingestion = "bkt-musicstream-bi/Files/TransientZone"
bucket_raw = "bkt-musicstream-bi/Files/RawZone"
bucket_trusted = "bkt-musicstream-bi/Files/TrustedZone"
bucket_refined = "bkt-musicstream-bi/Files/RefinedZone"

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [9]:
### CHANGE FOR EACH FILE
output_trusted = 'trusted_user_info'

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [10]:
full_path_raw = 's3://{bkt}/{file}'.format(bkt=bucket_raw,file=raw)
print(full_path_raw)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

s3://bkt-musicstream-bi/Files/RawZone/t_user_info

In [11]:
path_lake = os.path.join(full_path_raw)
raw_00=sqlContext.read.format("parquet").load(path_lake)
raw_00.registerTempTable("raw_00")

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [11]:
raw_00.printSchema()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

root
 |-- ts_proc: long (nullable = true)
 |-- user_id: string (nullable = true)
 |-- ts_proc_partition: long (nullable = true)

In [16]:
raw_00.show(10)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+--------------+--------------------+-----------------+
|       ts_proc|             user_id|ts_proc_partition|
+--------------+--------------------+-----------------+
|20230827151600|00000b72200188206...|   20230827151600|
|20230827151600|00001638d61892368...|   20230827151600|
|20230827151600|0000175652312d125...|   20230827151600|
|20230827151600|00001cf0dce3fb22b...|   20230827151600|
|20230827151600|0000267bde1b3a70e...|   20230827151600|
|20230827151600|00004fb90a86beb8b...|   20230827151600|
|20230827151600|00005c6177188f12f...|   20230827151600|
|20230827151600|000060ca4e6bea0a5...|   20230827151600|
|20230827151600|00007ed2509128dcd...|   20230827151600|
|20230827151600|00008c7cc5606ba35...|   20230827151600|
+--------------+--------------------+-----------------+
only showing top 10 rows

In [12]:
print('lista de colunas para tipar')
for col in raw_00.columns:
    print('cast(' + col + ' as) as ' + col + ',')

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

lista de colunas para tipar
cast(ts_proc as) as ts_proc,
cast(user_id as) as user_id,
cast(ts_proc_partition as) as ts_proc_partition,

In [13]:
lake = spark.sql(     
    """
        select
            -- padrao para todas as cargas no datalake--
            
            --ref,
            --ref_partition,
            
            {pdthproc} as ts_proc,
            {pdthproc} as ts_proc_partition,
        
            -- campos do arquivo --
            
            cast(user_id as string) as user_id
            
        from
            raw_00            
    """.format(pdthproc=dthproc))
lake.registerTempTable("lake")
lake.cache()
lake.count()  

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

962037

In [25]:
### CHANGE FOR EACH FILE
path_trusted = os.path.join('s3://',bucket_trusted,output_trusted)
print(path_trusted)

lake.coalesce(1).write.parquet(path_trusted,mode='overwrite')
#partitionBy("genre","ts_proc_partition"). \
#parquet(path_trusted,mode='overwrite')

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

s3://bkt-musicstream-bi/Files/TrustedZone/trusted_user_info

In [14]:
varDataFile = output_trusted

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [15]:
controle = spark.sql(    
        """
            select
                '{tb}' as name_file,
                '{tb}' as name_file_partition,
                
                --ref,
                --ref_partition,
                
                ts_proc,
                ts_proc_partition,
                
                count(*) as qtd_registros
            from 
                lake as a
            group by
                1,2,3,4
            order by
                1,2,3,4
        """.format(tb=varDataFile))

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [16]:
path_control = os.path.join('s3://',bucket_control,'tb_0002_controle_procesamento_trusted')
print(path_control)
controle.coalesce(1).write.\
parquet(path_control,mode='append')

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

s3://bkt-musicstream-bi/Files/Control/tb_0002_controle_procesamento_trusted

In [ ]:
# End